In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os

from model1 import Model1
from endovis_dataset import EndovisDataset
from reformat_data import convert_all_data

segmentation_model_weights_path = "model1_weights.pth"
tracking_model_weights_path = "tracking_model_weights.pth"

og_data_path = r"original_data"
data_path = r"data"

# Hyperparameters 
# TODO tune these! I just made them randomly
batch_size = 16
learning_rate = 0.0001
momentum = 0.9
epochs = 10




In [2]:
# Converts all the data from video to npz files
# Only need to run this once when setting up your system

# Subfolders where data is; True indicates it is segmentation data
subfolders = { 
    "segmentation_train": True,
    "segmentation_test": True,
    "tracking_train": False,
    "tracking_test": False
}
convert_all_data(og_data_path, data_path, subfolders)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '\\data'

In [3]:
# Training for Segmentation

# Model
model = Model1(4) # Switch out with whichever model you like

folder = os.path.join(data_path, "segmentation_train")
dataset = EndovisDataset(folder)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True) # TODO can tune batch size

criterion = nn.BCELoss() # Because https://medium.com/@kitkat73275/multi-label-classification-8d8ae55e8373#:~:text=For%20multi%2Dlabel%20classification%20with,class%20labels%20for%20new%20instances.
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum) # TODO tune these hyperparameters

for epoch in range(epochs):
    print("Epoch %d:" % epoch)
    for frames, truths in dataloader:
        optimizer.zero_grad()
        outputs = model(frames)
        loss = criterion(outputs, truths)
        loss.backward()
        optimizer.step()
    print("Loss: %f" % loss.item())

torch.save(model.state_dict(), segmentation_model_weights_path)


Epoch 0:
.
.
.
.
.


In [ ]:
# Testing for Segmentation

model = Model1()
model.load_state_dict(torch.load(segmentation_model_weights_path))
model.eval()

test_folder = r"data\segmentation_test"
testset = EndovisDataset(test_folder)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True) 

# Test the model
with torch.no_grad():
    for frames, truths in testloader:
        outputs = torch.sigmoid(model(frames)) # Apply sigmoid for probabilities
        # TODO Evaluate performance, like IOU or DICE

In [ ]:
# Training for Tracking

# TODO Cloud

In [ ]:
# Testing for Tracking

# TODO Cloud